# Decision Trees

In [1]:
# necessary imports
import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [2]:
hdfc_dataset = pd.read_csv("/Users/janhavi/Desktop/HDFC.csv", parse_dates=['Date'])
hdfc_dataset

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2000-01-03,HDFC,EQ,271.75,293.50,293.50,293.50,293.50,293.50,293.50,22744,6.675364e+11,NaN,NaN,NaN
1,2000-01-04,HDFC,EQ,293.50,317.00,317.00,297.00,304.00,304.05,303.62,255251,7.749972e+12,NaN,NaN,NaN
2,2000-01-05,HDFC,EQ,304.05,290.00,303.90,285.00,295.00,292.80,294.53,269087,7.925368e+12,NaN,NaN,NaN
3,2000-01-06,HDFC,EQ,292.80,301.00,314.00,295.00,296.00,296.45,300.14,305916,9.181669e+12,NaN,NaN,NaN
4,2000-01-07,HDFC,EQ,296.45,290.00,296.35,281.00,287.10,286.55,288.80,197039,5.690480e+12,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5301,2021-04-26,HDFC,EQ,2497.35,2500.00,2534.10,2483.20,2502.00,2509.80,2508.07,3916088,9.821805e+14,121028.0,2440395.0,0.6232
5302,2021-04-27,HDFC,EQ,2509.80,2494.15,2526.80,2486.25,2514.00,2518.40,2509.18,2040799,5.120730e+14,102250.0,1040749.0,0.5100
5303,2021-04-28,HDFC,EQ,2518.40,2516.10,2609.00,2508.30,2575.00,2577.00,2574.21,3407461,8.771527e+14,117425.0,1815110.0,0.5327
5304,2021-04-29,HDFC,EQ,2577.00,2590.90,2628.00,2533.00,2539.70,2538.85,2569.65,3005468,7.722995e+14,132826.0,1472924.0,0.4901


In [3]:
# Replacing space in columns name with a _ since pandas cannot handle it
hdfc_dataset.columns = [x.replace(' ', '_') for x in hdfc_dataset.columns]

In [4]:
# Replacing % in columns name with a _ 
hdfc_dataset.columns = [x.replace('%', '_') for x in hdfc_dataset.columns]
hdfc_dataset.head(5)

,Date,Symbol,Series,Prev_Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable_Volume,_Deliverble
0,2000-01-03,HDFC,EQ,271.75,293.5,293.50,293.5,293.5,293.50,293.50,22744,6.675364e+11,NaN,NaN,NaN
1,2000-01-04,HDFC,EQ,293.50,317.0,317.00,297.0,304.0,304.05,303.62,255251,7.749972e+12,NaN,NaN,NaN
2,2000-01-05,HDFC,EQ,304.05,290.0,303.90,285.0,295.0,292.80,294.53,269087,7.925368e+12,NaN,NaN,NaN
3,2000-01-06,HDFC,EQ,292.80,301.0,314.00,295.0,296.0,296.45,300.14,305916,9.181669e+12,NaN,NaN,NaN
4,2000-01-07,HDFC,EQ,296.45,290.0,296.35,281.0,287.1,286.55,288.80,197039,5.690480e+12,NaN,NaN,NaN


In [5]:
# Doing data cleaning by removing unnecessary columns which wont be used for predictions
drop_column = ['Date', 'Symbol', 'Series', 'Trades', 'Deliverable_Volume', '_Deliverble']
hdfc_dataset.drop(drop_column, axis = 1, inplace = True)

In [6]:
# remove the last observation (last row) sincethere is no tomorrow for the last row to predict
X = hdfc_dataset.iloc[0:-1]
X

y = hdfc_dataset.Close.shift(-1).dropna()
y

0        304.05
1        292.80
2        296.45
3        286.55
4        287.20
         ...   
5300    2509.80
5301    2518.40
5302    2577.00
5303    2538.85
5304    2420.10
Name: Close, Length: 5305, dtype: float64

In [7]:
# predict for next 250 days for HDFC dataset
L = hdfc_dataset.iloc[0:-250]
L

m = hdfc_dataset.Close.shift(-250).dropna()
m

0        538.25
1        542.15
2        552.15
3        550.60
4        583.30
         ...   
5051    2509.80
5052    2518.40
5053    2577.00
5054    2538.85
5055    2420.10
Name: Close, Length: 5056, dtype: float64

In [8]:
X.T

,0,1,2,3,4,5,6,7,8,9,...,5295,5296,5297,5298,5299,5300,5301,5302,5303,5304
Prev_Close,2.717500e+02,2.935000e+02,3.040500e+02,2.928000e+02,2.964500e+02,2.865500e+02,2.872000e+02,2.838500e+02,2.856000e+02,2.838500e+02,...,2.512950e+03,2.547150e+03,2.574050e+03,2.492350e+03,2.415900e+03,2.479700e+03,2.497350e+03,2.509800e+03,2.518400e+03,2.577000e+03
Open,2.935000e+02,3.170000e+02,2.900000e+02,3.010000e+02,2.900000e+02,2.920000e+02,2.900000e+02,2.870000e+02,2.880000e+02,2.840000e+02,...,2.540000e+03,2.550000e+03,2.474000e+03,2.542150e+03,2.400000e+03,2.455050e+03,2.500000e+03,2.494150e+03,2.516100e+03,2.590900e+03
High,2.935000e+02,3.170000e+02,3.039000e+02,3.140000e+02,2.963500e+02,2.960000e+02,2.920000e+02,2.930000e+02,2.905000e+02,2.940000e+02,...,2.555000e+03,2.589800e+03,2.510000e+03,2.544000e+03,2.485000e+03,2.504150e+03,2.534100e+03,2.526800e+03,2.609000e+03,2.628000e+03
Low,2.935000e+02,2.970000e+02,2.850000e+02,2.950000e+02,2.810000e+02,2.850000e+02,2.732500e+02,2.845000e+02,2.830000e+02,2.762500e+02,...,2.493000e+03,2.546850e+03,2.452000e+03,2.406200e+03,2.373000e+03,2.437500e+03,2.483200e+03,2.486250e+03,2.508300e+03,2.533000e+03
Last,2.935000e+02,3.040000e+02,2.950000e+02,2.960000e+02,2.871000e+02,2.884000e+02,2.828500e+02,2.852500e+02,2.840000e+02,2.910000e+02,...,2.543950e+03,2.569000e+03,2.501050e+03,2.409000e+03,2.478050e+03,2.491000e+03,2.502000e+03,2.514000e+03,2.575000e+03,2.539700e+03
Close,2.935000e+02,3.040500e+02,2.928000e+02,2.964500e+02,2.865500e+02,2.872000e+02,2.838500e+02,2.856000e+02,2.838500e+02,2.865500e+02,...,2.547150e+03,2.574050e+03,2.492350e+03,2.415900e+03,2.479700e+03,2.497350e+03,2.509800e+03,2.518400e+03,2.577000e+03,2.538850e+03
VWAP,2.935000e+02,3.036200e+02,2.945300e+02,3.001400e+02,2.888000e+02,2.894200e+02,2.845400e+02,2.876000e+02,2.858400e+02,2.836000e+02,...,2.527230e+03,2.572390e+03,2.484260e+03,2.454680e+03,2.437460e+03,2.479650e+03,2.508070e+03,2.509180e+03,2.574210e+03,2.569650e+03
Volume,2.274400e+04,2.552510e+05,2.690870e+05,3.059160e+05,1.970390e+05,1.333630e+05,3.374110e+05,2.225370e+05,1.132380e+05,1.523220e+05,...,4.201633e+06,3.133311e+06,3.550890e+06,7.365765e+06,4.514740e+06,3.143779e+06,3.916088e+06,2.040799e+06,3.407461e+06,3.005468e+06
Turnover,6.675364e+11,7.749972e+12,7.925368e+12,9.181669e+12,5.690480e+12,3.859779e+12,9.600617e+12,6.400217e+12,3.236741e+12,4.319905e+12,...,1.061850e+15,8.060093e+14,8.821343e+14,1.808059e+15,1.100450e+15,7.795465e+14,9.821805e+14,5.120730e+14,8.771527e+14,7.722995e+14


In [9]:
L.T

,0,1,2,3,4,5,6,7,8,9,...,5046,5047,5048,5049,5050,5051,5052,5053,5054,5055
Prev_Close,2.717500e+02,2.935000e+02,3.040500e+02,2.928000e+02,2.964500e+02,2.865500e+02,2.872000e+02,2.838500e+02,2.856000e+02,2.838500e+02,...,1.625750e+03,1.680600e+03,1.727700e+03,1.669700e+03,1.664150e+03,1.663450e+03,1.580300e+03,1.591450e+03,1.715800e+03,1.836750e+03
Open,2.935000e+02,3.170000e+02,2.900000e+02,3.010000e+02,2.900000e+02,2.920000e+02,2.900000e+02,2.870000e+02,2.880000e+02,2.840000e+02,...,1.690000e+03,1.709800e+03,1.670000e+03,1.669700e+03,1.674900e+03,1.603000e+03,1.618100e+03,1.615800e+03,1.734000e+03,1.857000e+03
High,2.935000e+02,3.170000e+02,3.039000e+02,3.140000e+02,2.963500e+02,2.960000e+02,2.920000e+02,2.930000e+02,2.905000e+02,2.940000e+02,...,1.738000e+03,1.738000e+03,1.694800e+03,1.679000e+03,1.687000e+03,1.624950e+03,1.652150e+03,1.725000e+03,1.845000e+03,1.927000e+03
Low,2.935000e+02,2.970000e+02,2.850000e+02,2.950000e+02,2.810000e+02,2.850000e+02,2.732500e+02,2.845000e+02,2.830000e+02,2.762500e+02,...,1.652400e+03,1.673400e+03,1.651100e+03,1.626150e+03,1.651600e+03,1.569100e+03,1.585350e+03,1.612650e+03,1.726000e+03,1.855500e+03
Last,2.935000e+02,3.040000e+02,2.950000e+02,2.960000e+02,2.871000e+02,2.884000e+02,2.828500e+02,2.852500e+02,2.840000e+02,2.910000e+02,...,1.680000e+03,1.734000e+03,1.665000e+03,1.670000e+03,1.656000e+03,1.588000e+03,1.600600e+03,1.724000e+03,1.827950e+03,1.916150e+03
Close,2.935000e+02,3.040500e+02,2.928000e+02,2.964500e+02,2.865500e+02,2.872000e+02,2.838500e+02,2.856000e+02,2.838500e+02,2.865500e+02,...,1.680600e+03,1.727700e+03,1.669700e+03,1.664150e+03,1.663450e+03,1.580300e+03,1.591450e+03,1.715800e+03,1.836750e+03,1.916000e+03
VWAP,2.935000e+02,3.036200e+02,2.945300e+02,3.001400e+02,2.888000e+02,2.894200e+02,2.845400e+02,2.876000e+02,2.858400e+02,2.836000e+02,...,1.688580e+03,1.709850e+03,1.676040e+03,1.659270e+03,1.666790e+03,1.599710e+03,1.620720e+03,1.671930e+03,1.803340e+03,1.894760e+03
Volume,2.274400e+04,2.552510e+05,2.690870e+05,3.059160e+05,1.970390e+05,1.333630e+05,3.374110e+05,2.225370e+05,1.132380e+05,1.523220e+05,...,8.954559e+06,7.372380e+06,6.604554e+06,5.301750e+06,4.419421e+06,7.166440e+06,5.719079e+06,7.691600e+06,1.188151e+07,8.237079e+06
Turnover,6.675364e+11,7.749972e+12,7.925368e+12,9.181669e+12,5.690480e+12,3.859779e+12,9.600617e+12,6.400217e+12,3.236741e+12,4.319905e+12,...,1.512048e+15,1.260565e+15,1.106948e+15,8.797053e+14,7.366264e+14,1.146423e+15,9.269017e+14,1.285981e+15,2.142640e+15,1.560732e+15


In [10]:
# Split the data into train and test data using the train)test_split API 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Split the data into train and test data using the train)test_split API 
L_train, L_test, m_train, m_test = train_test_split(L, m, random_state=42)

In [12]:
# Create a model 
rf_model = RandomForestRegressor()

# Train the model
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [13]:
# Create a model 
rf_model = RandomForestRegressor()

# Train the model
rf_model.fit(L_train, m_train)

RandomForestRegressor()

In [14]:
# Get R2 measure (indicator of accuracy 1 is perfect, 0 is bad)
rf_model.score(X_test, y_test)

0.3690580184690099

In [15]:
# Get R2 measure (indicator of accuracy 1 is perfect, 0 is bad)
rf_model.score(L_test, m_test)

0.8492164026062246